In [342]:
# to get JSON files from TOS API
import urllib, json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [119]:
#create a list of file names to pull each json file
from os import listdir
from os.path import isfile, join
companies = [f for f in listdir('tosdr.org/api/1/service') if isfile(join('tosdr.org/api/1/service', f))]

In [415]:
# nlp imports, chose TFIDF since some words are repeated
from sklearn.feature_extraction.text import TfidfVectorizer

In [244]:
#Pull out the data 
data_list = []

for company in companies:
    with open(f'tosdr.org/api/1/service/{company}') as json_data:
        data_list.append(json.load(json_data))

In [383]:
#Create main dataframe, drop unnecessary columns, expand list of lists

pd_list = []
for data in data_list:
    pd_list.append(pd.DataFrame.from_dict(data['pointsData']).T.loc[:,['id','quoteText','services','title','tosdr']])

alldata_df = pd.concat(pd_list,axis=0).reset_index()
todsr_df = json_normalize(alldata_df['tosdr'])
df = pd.concat((alldata_df,todsr_df),axis=1)
df = df.drop('tosdr',axis=1).explode('services')

In [392]:
df['document'] = df['tldr']

In [422]:
df['document'] = np.where(df['document'] == '',df['quoteText'],df['document'])
df['document'] = np.where(df['document'] == 'Generated through the annotate view',df['quoteText'],df['document'])
df['document'].fillna(df['quoteText'])

0       The terms of service state that "Signal does n...
1       The terms of service state: "You must be at le...
2       The terms of service state: "You agree to reso...
3       The terms of service state: "We may modify, su...
4       The privacy policy states: "We work with third...
                              ...                        
2559    Altogether, ToS and PP only make up eight line...
2560    The ToS state: "Nothing here should be conside...
2561    We do not use cookies or any other tracking te...
2562    By contributing to this project (e.g.\nby send...
2563    We need you to license your contributions unde...
Name: document, Length: 2564, dtype: object

In [400]:
df

,index,id,quoteText,services,title,binding,case,point,privacyRelated,score,tldr,sources,irrelevant,reason,tmp_rating,document
0,4411,4411,"Signal does not sell, rent or monetize your pe...",signal,This service does not sell your personal data,True,This service does not sell your personal data,good,True,25,"The terms of service state that ""Signal does n...",NaN,NaN,NaN,NaN,"The terms of service state that ""Signal does n..."
1,4412,4412,You must be at least 13 years old to use our S...,signal,You must be at least 13 years old to use the s...,True,This service is only available to users of a c...,neutral,NaN,15,"The terms of service state: ""You must be at le...",NaN,NaN,NaN,NaN,"The terms of service state: ""You must be at le..."
2,4414,4414,You agree to resolve any Claim you have with u...,signal,Court of law is in California,True,The court of law governing the terms is in loc...,neutral,NaN,0,"The terms of service state: ""You agree to reso...",NaN,NaN,NaN,NaN,"The terms of service state: ""You agree to reso..."
3,4415,4415,"We may modify, suspend, or terminate your acce...",signal,The service can delete your account without pr...,True,The service can delete your account without pr...,bad,NaN,60,"The terms of service state: ""We may modify, su...",NaN,NaN,NaN,NaN,"The terms of service state: ""We may modify, su..."
4,4416,4416,We work with third parties to provide some of ...,signal,Third parties may be involved in operating the...,True,Third parties may be involved in operating the...,bad,NaN,15,"The privacy policy states: ""We work with third...",NaN,NaN,NaN,NaN,"The privacy policy states: ""We work with third..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,4523,4523,NaN,tos;dr,The terms for ToS;DR are easy to read,True,The terms for this service are easy to read,good,NaN,15,"Altogether, ToS and PP only make up eight line...",NaN,NaN,NaN,NaN,"Altogether, ToS and PP only make up eight line..."
2560,4524,4524,Nothing here should be considered legal advice...,tos;dr,ToS;DR gives no guarantee regarding quality,True,This services gives no guarantee regarding qua...,bad,NaN,20,"The ToS state: ""Nothing here should be conside...",NaN,NaN,NaN,NaN,"The ToS state: ""Nothing here should be conside..."
2561,7679,7679,We do not use cookies or any other tracking te...,tos;dr,This service does not track you,True,This service does not track you,good,NaN,100,Generated through the annotate view,NaN,NaN,NaN,NaN,We do not use cookies or any other tracking te...
2562,7683,7683,By contributing to this project (e.g.\nby send...,tos;dr,"If you offer suggestions to the service, they ...",True,"If you offer suggestions to the service, they ...",good,NaN,50,Generated through the annotate view,NaN,NaN,NaN,NaN,By contributing to this project (e.g.\nby send...


In [404]:
df.loc[np.random.randint(1,2564,5),['quoteText','tldr','document']]

,quoteText,tldr,document
2095,NaN,"""If you wish to have your personal information...","""If you wish to have your personal information..."
2203,"Effective Date: January 13, 2005<br>Last Updat...",Generated through the annotate view,"Effective Date: January 13, 2005<br>Last Updat..."
1799,"Notwithstanding the foregoing, in no event sha...",The terms of conditions states that any liabil...,The terms of conditions states that any liabil...
818,use Brainly Services if we have suspended your...,"The Terms of Use (in Part B, applicable to use...","The Terms of Use (in Part B, applicable to use..."
725,To export the personal data you have provided ...,Generated through the annotate view,To export the personal data you have provided ...


In [423]:
df[df['document'].isna()]

,index,id,quoteText,services,title,binding,case,point,privacyRelated,score,tldr,sources,irrelevant,reason,tmp_rating,document
1079,1332,1332,NaN,travelzoo,Vouchers Can be Redeemed for Cash,True,Users are entitled to a refund if certain thre...,good,NaN,30,,NaN,NaN,NaN,NaN,NaN
1881,6913,6913,NaN,sourcehut,Features of the website are made available und...,True,Features of the website are made available und...,good,NaN,90,,NaN,NaN,NaN,NaN,NaN
1898,6074,6074,NaN,observable,This service does not force users into binding...,True,This service does not force users into binding...,good,NaN,50,,NaN,NaN,NaN,NaN,NaN
2534,4659,4659,NaN,openbookpublishers,The terms for this service are easy to read,True,The terms for this service are easy to read,good,NaN,15,,NaN,NaN,NaN,NaN,NaN


In [416]:
tfiddy = TfidfVectorizer()

In [417]:
tfiddy.fit(df['document'])

ValueError: np.nan is an invalid document, expected byte or unicode string.